**Loading images and labels**

In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from collections import Counter
import math
from sklearn.model_selection import train_test_split

def count_classes(data_dir):
    labels = []
    for label in os.listdir(data_dir):
        for image_file in os.listdir(os.path.join(data_dir, label)):
            labels.append(label)
    return Counter(labels)

def load_data(data_dir):
    images, labels = [], []
    for label in os.listdir(data_dir):
        for image_file in os.listdir(os.path.join(data_dir, label)):
            img = image.load_img(os.path.join(data_dir, label, image_file), target_size=(256, 256))
            img_tensor = image.img_to_array(img)
            img_tensor = np.expand_dims(img_tensor, axis=0)
            img_tensor /= 255.  # normalize to [0,1] range

            images.append(img_tensor)
            labels.extend([label] * len(img_tensor))

    return np.concatenate(images, axis=0), np.array(labels)

def augment_data(images, labels, class_counts):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True)

    augmented_images, augmented_labels = [], []
    max_images = max(class_counts.values())
    for img_tensor, label in zip(images, labels):
        aug_images = [img_tensor]
        augmentation_factor = max_images / class_counts[label]
        for batch in datagen.flow(np.expand_dims(img_tensor, axis=0), batch_size=1):
            aug_images.append(np.squeeze(batch, axis=0))
            if len(aug_images) >= math.ceil(20 * augmentation_factor):  # original + augmented images
                break
        aug_images = np.stack(aug_images, axis=0)  # stack images along a new axis
        augmented_images.append(aug_images)
        augmented_labels.extend([label] * len(aug_images))

    return np.concatenate(augmented_images, axis=0), np.array(augmented_labels)

# Load all data without augmentation
images, labels = load_data('ds')

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

class_counts = count_classes('ds')
# Augment the training data
# X_train, y_train = augment_data(X_train, y_train, class_counts)

print(X_train.shape, y_train.shape)
print(Counter(y_train))
print(X_test.shape, X_test.shape)
print(Counter(y_test))

(532, 256, 256, 3) (532,)
Counter({'Palm': 299, 'Fist': 160, 'Thumb': 73})
(134, 256, 256, 3) (134, 256, 256, 3)
Counter({'Palm': 75, 'Fist': 40, 'Thumb': 19})


**DS split**

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
y_val = le.transform(y_val)



**Model teaching**

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras import backend as K

K.clear_session(free_memory=True)

tf.random.set_seed(42)


model = Sequential()
model.add(Flatten(input_shape=(256, 256, 3)))
model.add(Dense(200, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='softmax')) 

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])


Epoch 1/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 280ms/step - accuracy: 0.3688 - loss: 14.5867 - val_accuracy: 0.2991 - val_loss: 6.1181
Epoch 2/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 265ms/step - accuracy: 0.4480 - loss: 4.3171 - val_accuracy: 0.6729 - val_loss: 0.9127
Epoch 3/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 260ms/step - accuracy: 0.6118 - loss: 1.1855 - val_accuracy: 0.6075 - val_loss: 1.0375
Epoch 4/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 261ms/step - accuracy: 0.6620 - loss: 0.9254 - val_accuracy: 0.7196 - val_loss: 0.6915
Epoch 5/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 261ms/step - accuracy: 0.8157 - loss: 0.4604 - val_accuracy: 0.7196 - val_loss: 0.5549
Epoch 6/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 266ms/step - accuracy: 0.8260 - loss: 0.3890 - val_accuracy: 0.7570 - val_loss: 0.5105
Epoch 7/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 266ms/step - accuracy: 0.8459 - loss: 0.3679 - val_accuracy: 0.8037 - val_loss: 0.6333
Epoch 8/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 263ms/step - accuracy: 0.8291 - loss: 0.4111 -

**Predict and evaluate**

In [4]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred)
unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

unique, counts = np.unique(y_test, return_counts=True)
print(dict(zip(unique, counts)))

print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred, target_names=le.classes_))



5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
{0: 128, 1: 239, 2: 58}
{0: 40, 1: 75, 2: 19}
Accuracy: 0.9328358208955224
              precision    recall  f1-score   support

        Fist       0.86      0.95      0.90        40
        Palm       0.96      0.97      0.97        75
       Thumb       1.00      0.74      0.85        19

    accuracy                           0.93       134
   macro avg       0.94      0.89      0.91       134
weighted avg       0.94      0.93      0.93       134



Save the Model

In [5]:
# Evaluate the model on the test set
_, accuracy = model.evaluate(X_test, y_test, verbose=0)

# Get the number of layers in the model
num_layers = len([layer for layer in model.layers if type(layer) is Dense])

# Create the filename
filename = f"only_dense_{num_layers}_layers_no_augmentation_{accuracy:.2f}accuracy.keras"

# Save the model
model.save(f"./saved_models/{filename}")

Load a model

In [5]:
import tensorflow as tf
from tensorflow.keras.models import load_model
filename = "only_dense_7_layers_0.91_accuracy.keras"
model = tf.keras.models.load_model(f"./saved_models/{filename}")

c:\DevTools\Python\lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 16 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Solution for second pipeline:


With same net as in first iteration:

One of my first runs had 50% but cant reproduce it
Only 3 Minimum Augmentations 256x256 ca. 35,82 Accuracy always pred palm
20 Minimum Augmentation 256x256 takes forever to load images, still 35,82

With deeper net: 

model = Sequential()
model.add(Flatten(input_shape=(256, 256, 3)))
model.add(Dense(200, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='softmax')) 

20 Minimum Augmentation 256x256, 91% accuracy -> saved this one

Try with even deeper net:

model = Sequential()
model.add(Flatten(input_shape=(256, 256, 3)))
model.add(Dense(200, activation='relu'))
model.add(Dense(175, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(125, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(75, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='softmax')) 

20 Minimum Augmentation 256x256, also 91% accuracy -> saved this one

Another try without Augmentation of test data and with dropout: 

model = Sequential()
model.add(Flatten(input_shape=(256, 256, 3)))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(150, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(25, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

20 Minimum Augmentation 256x256, doesnt get higher than 35% accuracy 

First net without Test_data augmentation:
model = Sequential()
model.add(Flatten(input_shape=(256, 256, 3)))
model.add(Dense(200, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='softmax')) 

20 Minimum Augmentation 256x256, 93%

Another try without augmenting any data, with "deeper net" model:
93% -> saved